In [0]:
from keras.layers import Dense, Dropout,  Conv2D, Input, Lambda, Flatten, TimeDistributed
from keras.layers import Add, Reshape, MaxPooling2D, Concatenate, Embedding, RepeatVector, Multiply
from keras.models import Model
from keras import backend as K

import numpy as np
from keras.datasets import mnist
from keras.models import Sequential
from keras.layers.core import Dense, Dropout, Activation
from keras.utils import np_utils
from keras.engine.topology import Layer
import tensorflow as tf

Using TensorFlow backend.


In [0]:
def main_attention(x, l=6*6, d=64*3 , dv=8*3, nv = 8):
   
    v = Dense(d, activation = 'relu')(x)
    k = Dense(d, activation = 'relu')(x)
    q = Dense(d, activation = 'relu')(x)
   
    v = Reshape([l,nv,dv])(v)  
    k = Reshape([l,nv,dv])(k)  
    q = Reshape([l,nv,dv])(q)
   
    new = Multiply()([q,k])
    new = Activation('softmax')(new)
   
    new1 = Multiply()([new,v])
    new1 = Reshape([l,d])(new1)
    out = Add()([new1,x])
   
    return out

In [0]:
def multi_head(l=6*6, d=64*3 , dv=8*3, nv = 8):
   
    input_img = Input(shape = (l,d))
   
    out_1 = main_attention(input_img, l=6*6, d=64*3 , dv=8*3, nv = 8)
    out_2 = main_attention(input_img, l=6*6, d=64*3 , dv=8*3, nv = 8)
    out = Concatenate(axis = 2)([out_1,out_2])
    att_model = Model(input = input_img, output = out)
    #print(att_model.summary())
    return att_model


In [0]:
nb_classes = 10
(X_train, y_train), (X_test, y_test) = mnist.load_data()
print("X_train original shape", X_train.shape)
print("y_train original shape", y_train.shape)
X_train = X_train.reshape(60000, 28,28,1)
X_test = X_test.reshape(10000, 28,28,1)
X_train = X_train.astype('float32')
X_test = X_test.astype('float32')
X_train /= 255
X_test /= 255
print("Training matrix shape", X_train.shape)
print("Testing matrix shape", X_test.shape)
Y_train = np_utils.to_categorical(y_train, nb_classes)
Y_test = np_utils.to_categorical(y_test, nb_classes)

11493376/11490434 [==============================] - 1s 0us/step
X_train original shape (60000, 28, 28)
y_train original shape (60000,)
Training matrix shape (60000, 28, 28, 1)
Testing matrix shape (10000, 28, 28, 1)


In [0]:
inp = Input((28, 28, 1), name='input_layer')
x = Conv2D(32,(2,2),activation='relu', padding='same')(inp)
x = MaxPooling2D(pool_size=(2, 2))(x)
x = Conv2D(64,(2,2),activation='relu')(x)
x = MaxPooling2D(pool_size=(2, 2), padding='same')(x)
x = Conv2D(64*3,(2,2),activation='relu')(x)
x = Reshape([6*6,64*3])(x)  
att = multi_head(l=6*6, d=64*3 , dv=8*3, nv = 8)
out = att(x)
out = Dense(32, activation = 'relu')(out)
#x = Reshape([6,6,32])(x)  
x = Flatten()(out)
x = Dense(256, activation='relu')(x)
x = Dense(10, activation='softmax')(x)

model = Model(input = inp, output = x)

/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:8: UserWarning: Update your `Model` call to the Keras 2 API: `Model(inputs=Tensor("in..., outputs=Tensor("co...)`
  
/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:16: UserWarning: Update your `Model` call to the Keras 2 API: `Model(inputs=Tensor("in..., outputs=Tensor("de...)`
  app.launch_new_instance()


In [0]:
model.summary()

Model: "model_2"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_layer (InputLayer)     (None, 28, 28, 1)         0         
_________________________________________________________________
conv2d_1 (Conv2D)            (None, 28, 28, 32)        160       
_________________________________________________________________
max_pooling2d_1 (MaxPooling2 (None, 14, 14, 32)        0         
_________________________________________________________________
conv2d_2 (Conv2D)            (None, 13, 13, 64)        8256      
_________________________________________________________________
max_pooling2d_2 (MaxPooling2 (None, 7, 7, 64)          0         
_________________________________________________________________
conv2d_3 (Conv2D)            (None, 6, 6, 192)         49344     
_________________________________________________________________
reshape_1 (Reshape)          (None, 36, 192)           0   

In [0]:
model.compile(loss='categorical_crossentropy', optimizer='adam',metrics=['accuracy'])

In [0]:
model.fit(X_train, Y_train,
              batch_size=128, 
              epochs=100,
              verbose=1,          
              validation_data=(X_test, Y_test)
             )

Instructions for updating:
Use tf.where in 2.0, which has the same broadcast rule as np.where



Train on 60000 samples, validate on 10000 samples
Epoch 1/100





60000/60000 [==============================] - 23s 380us/step - loss: 0.2040 - acc: 0.9390 - val_loss: 0.0728 - val_acc: 0.9755
Epoch 2/100
60000/60000 [==============================] - 8s 137us/step - loss: 0.0538 - acc: 0.9824 - val_loss: 0.0369 - val_acc: 0.9882
Epoch 3/100
60000/60000 [==============================] - 8s 136us/step - loss: 0.0355 - acc: 0.9886 - val_loss: 0.0428 - val_acc: 0.9856
Epoch 4/100
60000/60000 [==============================] - 8s 138us/step - loss: 0.0270 - acc: 0.9915 - val_loss: 0.0385 - val_acc: 0.9887
Epoch 5/100
60000/60000 [==============================] - 8s 137us/step - loss: 0.0222 - acc: 0.9928 - val_loss: 0.0332 - val_acc: 0.9900
Epoch 6/100
60000/60000 [==============================] - 8s 137us/step - loss: 0.0181 - acc: 0.9943 - val_loss: 0.0334 - val_acc: 0.9901
Epoch 7/100
6